# Tarea2: Implementación y ataque criptosistema Vigènere

# Módulos

In [3]:
from itertools import chain

# Alfabeto y diccionarios

In [4]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789;:,.-'
alphSpecials = {
            'Á' : 'A',
            'É' : 'E',
            'Í' : 'I',
            'Ó' : 'O',
            'Ú' : 'U',
            'Ä' : 'A',
            'Ë' : 'E',
            'Ï' : 'I',
            'Ö' : 'O',
            'Ü' : 'U',
            'Ñ' : 'GN'
}

A partir de este alfabeto, creamos diccionarios que nos permitan codificar cada elemento de éste asignando a cada caracter un número. Tambien se crea el diccionario opuesto para poder realizar la operación opuesta y obtener un caracter a partir de un número. 

In [5]:
number_letter_map = {number:letter for number,letter in enumerate(alphabet)}
letter_number_map = {letter:number for number, letter in enumerate(alphabet)}

# Normalización

Antes de codificar el mensaje y cifrarlo, es necesario normalizarlo para eliminar cualquier caracter que no pertenezca al diccionario que hemos declarado y obtener una entrada uniforme. 

In [6]:
class Message:

    def _flatten(self, listOfLists: list) -> list:
        """Flatten one level of nesting"""
        return chain.from_iterable(listOfLists)

    def _rBlanks(self, strng: str) -> str:
        """Removes blanks of a string strng and converts to uppercase"""
        return ''.join(strng.split()).upper()

    def _normalize(self, strng: str) -> list:
        """
        Removes blanks spaces of the string 'strng'; then removes accents
        according to 'alphSpecials'. If character 'ñ' occurs in 'strng' then
        'GN' appears in 'accum' as an entry, therefore '_flatten' is needed.
        """
        s = self._rBlanks(strng)
        # s = [ch for ch in self._rBlanks(strng) if ch in alphabet]
        accum = []
        for ch in s:
            if ch in alphSpecials:
                accum.append(alphSpecials[ch])
            else:
                accum.append(ch)
        return filter(lambda x: x in alphabet, self._flatten(accum))
        # return [c for c in self._flatten(accum) if c in alphabet]

    def __init__(self, strng): #constructor los atributos son publicos
        x = self._normalize(strng)
        self.content = ''.join(x)
        self.length = len(self.content)

    def __str__(self):
        return self.content

La clase Message, con la funcion normalize, elimina los espacios entre las palabras, convierte todas las letras a mayusculas, elimina los caracteres especiales (sustituye las letras acentuadas por la letra sin acentuar y la Ñ por GN) y quita un nivel de anidamiento para que GN se pueda separar en caracteres individuales. 

# Cifrado

El cifrado de Vigenère funciona utilizando diferentes cifrados por desplazamiento para cifrar distintas letras. Para decidir cuánto desplazar cada letra, en primer lugar se debe acordar una palabra clave. Para cifrar entonces se usan las letras de la palabra clave, una por una, para determinar cuánto desplazar cada letra sucesiva del texto llano. Si la letra de la palabra clave es a, no hay desplazamiento; si la letra es b, desplaza en 1; si la letra es c, desplaza en 2; y así sucesivamente.

En primer lugar debemos establecer una palabra clave, para mayor seguridad se debe elegir una clave larga y con caracteres lo mas aleatorio posible, pero por simplificar eligiremos la clave "HOLA" 

In [7]:
key = "HOLA"

message = """El amanecer sobre la bahía fue lento y perezoso. Las luces de la ciudad aún parpadeaban en la distancia cuando las primeras embarcaciones comenzaron a cortar la niebla con su estela. Había un olor a sal y a algas que se mezclaba con el café recién hecho de los puestos del muelle, donde pescadores y comerciantes intercambiaban saludos y pequeñas historias de la noche anterior. Entre los muros de piedra, las gaviotas reclamaban su porción de jornada mientras los faros, como centinelas, guiaban a quien necesitase volver a puerto.

En el centro de la ciudad, la plaza mayor despertaba con un murmullo constante: vendedores acomodaban toldos, bicicletas cruzaban en línea recta sorteando charcos y grupos de estudiantes discutían proyectos y exámenes con entusiasmo contenido. Las fachadas, algunas recién pintadas y otras con la pátina del tiempo, contaban silenciosamente la historia de generaciones que habían pasado por las mismas esquinas. Un anciano, sentado en un banco con un periódico doblado, observaba el flujo de transeúntes con la calma de quien ha aprendido a distinguir lo urgente de lo verdaderamente importante.

Al otro extremo, un barrio industrial dejaba tras de sí el eco de máquinas y trabajo. Allí, las naves abandonadas habían sido reutilizadas como talleres, estudios y pequeños teatros de calle. Artistas y artesanos transformaban materiales olvidados en objetos con alma: esculturas metálicas que, al reflejar la luz, competían con las ventanas modernas que se alzaban a modo de testigos del cambio. En esos talleres se tejían también conversaciones sobre el futuro de la ciudad, sobre cómo conciliar la memoria colectiva con la atracción de la innovación.

En un parque cercano, niños y niñas corrían detrás de una pelota mientras sus familias extendían mantas y compartían comidas sencillas. Había quienes trajeron libros, quienes descansaban con los ojos cerrados y quienes aprovechaban para enseñar a los más pequeños a reconocer ciertas aves o a plantar una pequeña semilla en un macetero. La naturaleza, aun en su forma más domesticada, servía de recordatorio: los días son ciclos, y cada estación trae su pauta de trabajo y recuperación.

Más tarde, en la periferia, emergió la figura de un grupo de voluntarios que trabajaba en un huerto comunitario. Con guantes y regaderas, removían la tierra y conversaban sobre rotaciones de cultivos, plagas y técnicas naturales para mejorar la salud del suelo. Compartían semillas, herramientas y saberes: generaciones mayores explicaban trucos heredados y jóvenes introducían métodos modernos de compostaje y riego eficiente. El esfuerzo colectivo no sólo alimentaba estómagos, sino que también fortalecía lazos sociales y la sensación de pertenencia.

Al caer la tarde, la ciudad se transformó de nuevo: las luces se encendieron una a una, los cafés se llenaron y la música, que por la mañana había sido un susurro, ahora marcaba el ritmo de encuentros. Ciertas calles se volvieron íntimas; en otras, el bullicio anunciaba festivales improvisados. Las conversaciones se hicieron más profundas, las risas más liberadas y las miradas, a veces, más contemplativas. Cada persona llevaba consigo el día: triunfos, pequeñas derrotas, aprendizajes y la certeza de que mañana habrá otra oportunidad para empezar.

En las horas nocturnas, cuando la calma parecía reinar, se percibía un latido sutil: la ciudad respiraba y, en su respiro, sostenía promesas y proyectos. Había planes que quedaban en papel y otros que ya estaban en marcha; encuentros que cerraban ciclos y otros que sólo comenzaban. La vida urbana mostraba su faceta más compleja, pero también su capacidad para regenerarse, para acoger diferencias y para seguir adelante, un día tras otro.
"""

Normalizamos el mensaje que queremos cifrar con los metodos explicados anteriormente

In [8]:
class Encipher(Message): #la clase Encipher extiende la clase Message, heredando todos los metodos y atributos de Message
    def __init__(self, strng, key):
        Message.__init__(self, strng)
        self.key = key
        self.key_index = 0

    def encoder(self, char):
        return letter_number_map[char]

    def encrypt(self, char):
        encoded_char = self.encoder(char)
        ciphered_num = letter_number_map[self.key[self.key_index]] + encoded_char
        self.key_index = (self.key_index+1)%len(self.key)
        return alphabet[ciphered_num%len(alphabet)]

    def vigenere(self):
        cipher = [self.encrypt(c) for c in self.content]
        return ''.join(cipher)


El cifrado se ha implementado con un bucle que recorre cada caracter del mensaje. Cada caracter se codifica y se le suma la codificación de la letra en la que se encuentre el índice de la llave. A esta cifra se le aplica el módulo del tamaño del alfabeto para después descodificarla y obtener el caracter cifrado resultante, que se añadirá al mensaje final codificado. Al final de cada iteración aumentamos el índice de la llave una unidad (módulo la longitud de la llave) para utilizar la letra siguiente en la iteración posterior. 

In [9]:
P = Encipher(message, key)
ciphered_message = P.vigenere()
print(ciphered_message)

LZLMH1PCL53OI5PLHPLHPOQULZPN029PL5PZV6Z.SO3L1QPSKSWAJW5DHRLUU3LRWOOEHPLNL1WAKW3TH1NIHQ5AURZLH60RP0PRH6PMIO2CHQTOUS3CV0PN6O2OUONOY7LRSOYILPWAJ2YS1S3TLZL.OOMIH8YOS22AZOWYHOWGH61UL6PML.NLHPLCV1PLJOQEYSNIL1SEJVZDLZZSW8PS023DLZXULZWEERZNKS0EZQLDV5PS5QZML5NIH14EZWYTL5NATPTAIOYSHZ5DV69PL45EN1LSOW3TV5TAZRPLH1ZCOSLN0S2IV5JEU72ES23M15ZSKS0ILR2AEZLSNO6IV7LSYSNLH0LBH13UW22CP2YDLXZRUOOATWPN05LSS23FH5ZSEQZMVQPN0WYESO3,N8TAIOYAX8TEU1PCL6TTH6PVVZ6EYO0UL54OFSYESQPN05ZDLZLCP8OAKLWAWZLZH0LYV5OEZ3PR0OMAJ2YUU05RT8WLVQZNZ7LN0SHVL1OEK22EZONOT2OAIOYTVZOOZLMIJWNLL7LSJ55ZHPLNL1WIUSLRLQ4AZ22TLOYDVQSAYQZS5U2UW23DLS3T1RTAU7PSKW3C17TAU32O5SNTV69E4OXEUS3CV1PN083IH6XOJ2YTL1TDVMWAZTLCOOOAZLLLN8YAZ5PCPSYPP14AKO3YV72AZQZNSO0A0WYAKSWTPSXPVLNOU7LBH13ISSYCP23ATSYTLZLHP64OYWLDLUPNL5LCP2YEZ45EOOMIH10AZOOOW22LH6XIZ0LSL61UP1LSF8YAUQTAU2ISL14AK2PN11MAUQZCV15NWS2IVRTCVRZBSOOOE2MSL56AIOPLMZ5JVRPTYOYSL8YTL6NOUZLCHZXAKS1UPSYHHO0RL1OIK2LDP64IUU5IYZZUYUPN0SOES26EYRLDL5LML14EP00OY7LN0SJAS24RVS8TYSXOE8YBH52IVWYD164RPOWDLXLBH72AZRPSPSWE

Para descifrar el mensaje con la clave, unicamente tenemos que realizar la operacion contraria. Es decir, a cada caracter del mensaje cifrado se le resta la codificacion de la letra de la llave que le corresponda. 

In [10]:
class Decipher(): #la clase Encipher extiende la clase Message, heredando todos los metodos y atributos de Message
    def __init__(self, strng, key):
        self.strng = strng
        self.key = key
        self.key_index = 0

    def encoder(self, char):
        return letter_number_map[char]

    def decrypt(self, char):
        encoded_char = self.encoder(char)
        ciphered_num = encoded_char - letter_number_map[self.key[self.key_index]]
        self.key_index = (self.key_index+1)%len(self.key)
        return alphabet[ciphered_num%len(alphabet)]

    def vigenere(self):
        cipher = [self.decrypt(c) for c in self.strng]
        return ''.join(cipher)

In [11]:
Q = Decipher(ciphered_message, 'HOLA')
decipher = Q.vigenere()
print(decipher)

ELAMANECERSOBRELABAHIAFUELENTOYPEREZOSO.LASLUCESDELACIUDADAUNPARPADEABANENLADISTANCIACUANDOLASPRIMERASEMBARCACIONESCOMENZARONACORTARLANIEBLACONSUESTELA.HABIAUNOLORASALYAALGASQUESEMEZCLABACONELCAFERECIENHECHODELOSPUESTOSDELMUELLE,DONDEPESCADORESYCOMERCIANTESINTERCAMBIABANSALUDOSYPEQUEGNASHISTORIASDELANOCHEANTERIOR.ENTRELOSMUROSDEPIEDRA,LASGAVIOTASRECLAMABANSUPORCIONDEJORNADAMIENTRASLOSFAROS,COMOCENTINELAS,GUIABANAQUIENNECESITASEVOLVERAPUERTO.ENELCENTRODELACIUDAD,LAPLAZAMAYORDESPERTABACONUNMURMULLOCONSTANTE:VENDEDORESACOMODABANTOLDOS,BICICLETASCRUZABANENLINEARECTASORTEANDOCHARCOSYGRUPOSDEESTUDIANTESDISCUTIANPROYECTOSYEXAMENESCONENTUSIASMOCONTENIDO.LASFACHADAS,ALGUNASRECIENPINTADASYOTRASCONLAPATINADELTIEMPO,CONTABANSILENCIOSAMENTELAHISTORIADEGENERACIONESQUEHABIANPASADOPORLASMISMASESQUINAS.UNANCIANO,SENTADOENUNBANCOCONUNPERIODICODOBLADO,OBSERVABAELFLUJODETRANSEUNTESCONLACALMADEQUIENHAAPRENDIDOADISTINGUIRLOURGENTEDELOVERDADERAMENTEIMPORTANTE.ALOTROEXTREMO,UNBARRIOINDUSTRIALDEJABATRASDESIELE

# Ataque

Los métodos utilizados para el ataque al sistema de cifrado de Vigenère han sido obtenidos del articulo de  Mehmet E. Dalkilic y Cengiz Gungor llamado "An Interactive Cryptanalysis Algorithm for the Vigenere Cipher". También se han utilizado formulas para obtener distintos datos necesarios del libro "An Introduction to Mathematical Cryptography" escrito por los autores Jeffrey Hoffstein, Jill Pipher y Joseph H. Silverman.

El primer paso en el criptoanalisis consiste en encontrar la longitud de la llave. Para ello podemos emplear el test de Kasiski. Este test se basa en la observación de que a lo largo del texto cifrado, es posible ver fragmentos de tres o mas letras que aparecen repetidamente. Debido a la naturaleza cíclica del cifrado de Vigenère y al uso repetido de ciertas palabras, es probable que se correspondan al mismo texto plano y que hayan sido cifradas con la misma letra de la llave. Por lo tanto si obtenemos una lista de las distancias que separan a estas repeticiones, estas distancias probablemente seran un múltiplo de la longitud de la llave. En primer lugar definimos las siguientes funciones que nos seran útiles para implementar el test. 

In [12]:
import re
import math
from statistics import mean
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
number_letter_map = {number:letter for number,letter in enumerate(alphabet)}
letter_number_map = {letter:number for number, letter in enumerate(alphabet)}
SPANISH_INDEX_COINCIDENCE = 0.07

In [13]:
def possible_lengths(most_repeated):
    starting_positions = [m.start() for m in re.finditer(most_repeated[0], string)]
    distances = []
    for i in range(len(starting_positions) - 1):
        distances.append(starting_positions[i + 1] - starting_positions[i])
    #las longtides posibles del numero seran multiplos del mcd de las distancias.
    gcd = math.gcd(*distances)
    lengths = factorize(gcd)
    return lengths

def factorize(n):
    i = 2;
    number = n
    prime_numbers = []
    while i * i <= (n):
        if n % i == 0:
            prime_numbers.append(i)
            n = n // i
            while n % i == 0:
                n = n // i
        i = i + 1
    if n > 1: prime_numbers.append(n)
    if len(prime_numbers) > 1: prime_numbers.insert(0,n) #el propio numero tambien puede ser una posibilidad
    return prime_numbers

La función possible_lengths utiliza herramientas de la biblioteca re para localizar repeticiones de una cadena en el texto. La función finditer devuelve un iterador con todas las posiciones donde aparece la subsecuencia most_repeated[0]. A partir de esas coincidencias extraemos sus posiciones iniciales dentro del texto.
Luego calculamos las diferencias entre posiciones consecutivas, obteniendo un vector con las distancias entre repeticiones. Dado que estas distancias deben ser múltiplos de la longitud de la clave, calculamos el MCD de todas ellas mediante math.gcd. Finalmente, factorizamos ese MCD para obtener todas las posibles longitudes de la clave y las ordenamos de mas larga a mas corta ya que es mas probable que se elija una clave de mayor longitud al codificar.

In [14]:
def kasiski_test(string):
    # recorremos todo el texto buscando segmentos repetidas de longitud 3
    mapa = {} #los segmentos repetidos se colocan en el mapa y se contabiliza cuantas veces aparecen
    for i in range(len(string) - 2):
        if mapa.get(string[i:i + 3]) is None:
            mapa[string[i:i + 3]] = 1
        else:
            mapa[string[i:i + 3]] += 1

    #filtramos la lista de frases repetidas para quedarnos solo con las que se repitan más de 2 veces
    trigrams = list((filter(lambda x: x[1] > 2, mapa.items())))

    #ordenamos la lista para quedarnos con el más repetido  primero
    sorted_trigrams = sorted(trigrams, key = lambda x : x[1], reverse = True)
    
    lengths = []
    if(len(sorted_trigrams) > 0):
        most_repeated = sorted_trigrams[0] #el trigram mas repetido
        lengths = possible_lengths(most_repeated) #las posibles longitudes de la palabra clave
        print(lengths)
    else: #no hay elementos que se repitan
        print("La longitud del texto es demasiado corta, no se puede descifrar con el test de kasiski")

    return lengths

Un ejemplo lo podemos obtener con el siguiente texto codificado: 

In [15]:
string = (
"UECWKDVLOTTVACKTPVGEZQMDAMRNPDDUXLBUICAMRHOECBHSPQLVIWO"
"FFEAILPNTESMLDRUURIFAEQTTPXADWIAWLACCRPBHSRZIVQWOFROGTT"
"NNXEVIVIBPDTTGAHVIACLAYKGJIEQHGECMESNNOCTHSGGNVWTQHKBPR"
"HMVUOYWLIAFIRIGDBOEBQLIGWARQHNLOISQKEPEIDVXXNETPAXNZGDX"
"WWEYQCTIGONNGJVHSQGEATHSYGSDVVOAQCXLHSPQMDMETRTMDUXTEQQ"
"JMFAEEAAIMEZREGIMUECICBXRVQRSMENNWTXTNSRNBPZHMRVRDYNECG"
"SPMEAVTENXKEQKCTTHSPCMQQHSQGTXMFPBGLWQZRBOEIZHQHGRTOBSG"
"TATTZRNFOSMLEDWESIWDRNAPBFOFHEGIXLFVOGUZLNUSRCRAZGZRTTA"
"YFEHKHMCQNTZLENPUCKBAYCICUBNRPCXIWEYCSIMFPRUTPLXSYCBGCC"
"UYCQJMWIEKGTUBRHVATTLEKVACBXQHGPDZEANNTJZTDRNSDTFEVPDXK"
"TMVNAIQMUQNOHKKOAQMTBKOFSUTUXPRTMXBXNPCLRCEAEOIAWGGVVUS"
"GIOEWLIQFOZKSPVMEBLOHLXDVCYSMGOPJEFCXMRUIGDXNCCRPMLCEWT"
"PZMOQQSAWLPHPTDAWEYJOGQSOAVERCTNQQEAVTUGKLJAXMRTGTIEAFW"
"PTZYIPKESMEAFCGJILSBPLDABNFVRJUXNGQSWIUIGWAAMLDRNNPDXGN"
"PTTGLUHUOBMXSPQNDKBDBTEECLECGRDPTYBVRDATQHKQJMKEFROCLXN"
"FKNSCWANNAHXTRGKCJTTRRUEMQZEAEIPAWEYPAJBBLHUEHMVUNFRPVM"
"EDWEKMHRREOGZBDBROGCGANIUYIBNZQVXTGORUUCUTNBOEIZHEFWNBI"
"GOZGTGWXNRHERBHPHGSIWXNPQMJVBCNEIDVVOAGLPONAPWYPXKEFKOC"
"MQTRTIDZBNQKCPLTTNOBXMGLNRRDNNNQKDPLTLNSUTAXMNPTXMGEZKA"
"EIKAGQ"
)

In [16]:
possible_lengths = kasiski_test(string)
print(possible_lengths)

[7]
[7]


En este caso la única posibilidad que tenemos es que la clave del texto sea de longitud 7. 

$$
IC = \frac{\sum_{i=0}^{25} f_i(f_i - 1)}{n(n-1)}
$$
Donde: 
* $f_i$ es la frequencia absoluta de la letra i en el texto
* $n$ es la longitud del texto

In [17]:
frequencies = {} #mapa de las frecuencias absolutas de cada letra en cada substring
substrings = [] #array de todos los substrings, se utilizará para calculos posteriores

In [18]:
def data_build(length, message):# funcion auxiliar que calcula los datos necesarios para aplicar la formula
    for i in range(length):
        substring_i = []
        map_i = {}
        for j in range(i, len(message),length):
            character = message[j]
            substring_i.append(message[j]) #construimos el substring a la vez que calculamos las frecuencias absolutas de cada letra para ese substring
            if map_i.get(character) is None:
                map_i[character] = 1
            else:
                map_i[character] += 1
        substrings.append(substring_i)
        frequencies[i] = map_i

In [19]:
data_build(7, string)
print(len(substrings))
print(substrings)
print(frequencies)

7
[['U', 'L', 'K', 'Q', 'P', 'U', 'O', 'Q', 'F', 'T', 'U', 'Q', 'W', 'C', 'Z', 'R', 'X', 'P', 'V', 'K', 'G', 'N', 'G', 'K', 'U', 'F', 'O', 'W', 'O', 'E', 'E', 'G', 'Q', 'N', 'G', 'G', 'Q', 'Q', 'T', 'Q', 'E', 'R', 'C', 'Q', 'W', 'N', 'V', 'G', 'T', 'K', 'C', 'G', 'G', 'O', 'G', 'T', 'F', 'W', 'N', 'H', 'V', 'U', 'G', 'F', 'Q', 'P', 'C', 'P', 'C', 'U', 'C', 'C', 'K', 'V', 'V', 'G', 'N', 'N', 'P', 'N', 'N', 'Q', 'S', 'T', 'C', 'O', 'V', 'W', 'K', 'L', 'C', 'J', 'U', 'C', 'W', 'Q', 'P', 'J', 'V', 'Q', 'K', 'T', 'W', 'K', 'C', 'P', 'V', 'Q', 'W', 'N', 'P', 'U', 'Q', 'T', 'G', 'V', 'K', 'R', 'K', 'N', 'K', 'U', 'E', 'P', 'U', 'F', 'W', 'E', 'R', 'I', 'Q', 'U', 'O', 'W', 'G', 'H', 'G', 'Q', 'E', 'G', 'W', 'K', 'T', 'K', 'O', 'R', 'K', 'S', 'P', 'K', 'Q'], ['E', 'O', 'T', 'M', 'D', 'I', 'E', 'L', 'E', 'E', 'U', 'T', 'I', 'R', 'I', 'O', 'E', 'D', 'I', 'G', 'E', 'O', 'N', 'B', 'O', 'I', 'E', 'A', 'I', 'I', 'T', 'D', 'C', 'G', 'E', 'S', 'C', 'M', 'M', 'Q', 'A', 'E', 'I', 'R', 'T', 'B', 'R', 'S',

In [20]:
frequencies = {} #mapa de las frecuencias de cada substring
substrings = [] #array de todos los substrings

def index_of_coincidence_test(length, message):

    data_build(length, message)
    
    index_coincidence = [] #guarda el índice de coincidencia de cada subcadena
    for i in range (length): #para cada subcadena
        acc = 0
        num_letters = 0
        for letter in alphabet:
            frequency = frequencies[i].get(letter,0)
            acc += frequency * (frequency -1)
            num_letters += frequency
        index_coincidence.append((1/(num_letters*(num_letters-1))) * acc)

    return index_coincidence

Así, observamos que los índices de coincidencia para las subcadenas creadas a partir de coger cada séptima letra del texto son elevados y similares a el índice de coincidencia del español

In [21]:
frequencies = {} #mapa de las frecuencias de cada substring
substrings = [] #array de todos los substrings
indexes = index_of_coincidence_test(7, string)
print(indexes)

[0.0640176600441501, 0.07015659955257271, 0.06872483221476511, 0.06899328859060402, 0.07409395973154363, 0.07910514541387025, 0.07096196868008948]


Si por el contrario probamos el test con una longitud de cadena 5, estas subcadenas no tendrán la misma distribución de letras que el español y por lo tanto su indice de coincidencia será menor. 

In [22]:
frequencies = {} #mapa de las frecuencias de cada substring
substrings = [] #array de todos los substrings
indexes = index_of_coincidence_test(5, string)
print(indexes)

[0.03913337846987136, 0.04256094782410572, 0.04219640009113693, 0.04174071542492595, 0.043426748689906584]


In [23]:
frequencies = {} #mapa de las frecuencias de cada substring
substrings = [] #array de todos los substrings
indexes = index_of_coincidence_test(7, string)
if mean(indexes) > SPANISH_INDEX_COINCIDENCE: print("La longitud escogida es una buena aproximación \n")

La longitud escogida es una buena aproximación 



En caso de que la media no fuera mayor que el indice de coincidencia del español, tendríamos que probar con la siguiente longitud mas probable según el test de kasiski. Una vez tenemos calculada la longitud de la clave, podemos calcular la palabra empleando el Mutual Index of Coincidence (MIC). EL MIC de dos conjuntos a y b es la probabilidad de que al elegir un elemento de cada conjunto de forma aleatoria, estos elementos sean iguales. El MIC de a y b se puede calcular cómo: 
$$
MIC(\mathbf{a}, \mathbf{b}) =
\frac{\sum_{i=1}^{n} r_i\, r'_i}{l\, l'}
\;\simeq\;
\frac{\sum_{i=1}^{n} p_i\, r'_i}{l'}
$$
Donde: 
* $ r_i $ es número de apariciones de la letra $i$ en el texto $a$
* $ r'_i $ es el número de apariciones de la letra $i$ en el texto $b$
* $l$ es la longitud del lexto $a$
* $l'$ es la longitud del texto $b$
* $p_i$ es la frequencia teórica de la letra i en el idioma

Para una $l$ grande, la frequencia de aparición de la letra $i$ sera parecida a la frequencia teórica de esa letra en el idioma. Las frequencias teóricas de las letras en el idioma español son las siguientes: 

In [24]:
    p = {
        'A': 0.125125, 'B': 0.014181, 'C': 0.046735, 'D': 0.058518, 'E': 0.136609,
        'F': 0.006890, 'G': 0.010086, 'H': 0.006990, 'I': 0.062413, 'J': 0.004394,
        'K': 0.000200, 'L': 0.049631, 'M': 0.031456, 'N': 0.066999, 'O': 0.086679,
        'P': 0.025065, 'Q': 0.008788, 'R': 0.068605, 'S': 0.079689, 'T': 0.046235,
        'U': 0.039244, 'V': 0.008988, 'W': 0.000100, 'X': 0.002197, 'Y': 0.008988,
        'Z': 0.005193
    }

Dividimos el texto cifrado en m subcadenas, cada una cifrada con un desplazamiento fijo desconocido. Para recuperar cada desplazamiento, probamos a desplazar la subcadena por todas las posiciones posibles del alfabeto.

Para cada desplazamiento $b$ generamos la versión desplazada de la subcadena $s_j^{(b)}
$ y calculamos el MIC entre $s_j^{(b)}
$ y la distribucion teórica del idioma. El desplazamiento $b$ que produce el MIC maximo es el inverso aditivo de la clave real para esa subcadena. 

Por ejemplo si el máximo ocurre en $b = 5$ la clave es $-5 mod 26 = 21 $
Repitiendo esto para las m subcadenas obtenemos todas las letras de la clave. Si al descifrar aparece un texto coherente, la clave es correcta.

In [25]:
def mutual_index_coincidence():
    final_word = [] #lista que contendra cada letra de la clave

    for substring in substrings: #para cada substring...
        result_map = {} # mapa con el MIC resultante de probar cada desplazamiento
        best_MIC = 0 
        best_shift = 0 #el desplazamiento que ha causado el mayor MIC

        for i in range(len(alphabet)): #probamos cada desplazamiento...
            frequency_map = {}
            for letter in substring:
                #desplazamos cada letra del substring
                associated_number = letter_number_map[letter]
                new_number = (associated_number + i) % len(alphabet)
                associated_letter = number_letter_map[new_number]
                #calculamos las nuevas frecuencias de cada letra
                if frequency_map.get(associated_letter) is None:
                    frequency_map[associated_letter] = 1
                else: frequency_map[associated_letter] += 1
            MIC = 0
            f = 0
            for letter in alphabet:
                if frequency_map.get(letter) is None:
                    f = 0
                else: f = frequency_map[letter]
                MIC += p[letter] * (f / len(substring)) #calculo del MIC

            result_map[i] = MIC
            if MIC > best_MIC:
                best_MIC = MIC
                best_shift = i #nos quedamos con el mejor MIC

        final_word.append(alphabet[-best_shift % len(alphabet)])  #aplicamos el modulo del inverso additivo del mejor desplazamiento
    return final_word

In [26]:
lista = mutual_index_coincidence()
key = "".join(lista)
print(key)

CAPITAN


Finalmente, podemos descifrar el texto utilizando la clase definida anteriormente

In [27]:
Q = Decipher(string, key)
decipher = Q.vigenere()
print(decipher)

SENORDIJOELCAPITANNEMOMOSTRANDOMELOSINSTRUMENTOSCOLGADOSDELASPAREDESDESUCAMAROTEHEAQUILOSAPARATOSEXIGIDOSPORLANAVEGACIONDELNAUTILUSALIGUALQUEENELSALONLOSTENGOAQUIBAJOMISOJOSINDICANDOMEMISITUACIONYMIDIRECCIONEXACTASENMEDIODELOCEANOALGUNOSDEELLOSLESONCONOCIDOSCOMOELTERMOMETROQUEMARCALATEMPERATURAINTERIORDELNAUTILUSELBAROMETROQUEPESAELAIREYPREDICELOSCAMBIOSDETIEMPOELHIGROMETROQUEREGISTRAELGRADODESEQUEDADDELAATMOSFERAELSTORMGLASSCUYAMEZCLAALDESCOMPONERSEANUNCIALAINMINENCIADELASTEMPESTADESLABRUJULAQUEDIRIGEMIRUTAELSEXTANTEQUEPORLAALTURADELSOLMEINDICAMILATITUDLOSCRONOMETROSQUEMEPERMITENCALCULARMILONGITUDYPORULTIMOMISANTEOJOSDEDIAYDENOCHEQUEMESIRVENPARAESCRUTARTODOSLOSPUNTOSDELHORIZONTECUANDOELNAUTILUSEMERGEALASUPERFICIEDELASAGUASSONLOSINSTRUMENTOSHABITUALESDELNAVEGANTEYSUUSOMEESCONOCIDOREPUSEPEROHAYOTROSAQUIQUERESPONDENSINDUDAALASPARTICULARESEXIGENCIASDELNAUTILUSESECUADRANTEQUEVEORECORRIDOPORUNAAGUJAINMOVILNOESUNMANOMETROESUNMANOMETROENEFECTOPUESTOENCOMUNICACIONCONELAGUACUYAPRESIONEXTERIORIN